<a href="https://colab.research.google.com/github/mshumer/gpt-prompt-engineer/blob/main/gpt_prompt_engineer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Prompt Engineering 
Original Notebook By Matt Shumer (https://twitter.com/mattshumer_)

Original Github repo: https://github.com/mshumer/gpt-prompt-engineer

In this notebook, we will simulate how to generate an optimal prompt/instruction for a given task.

## Installations

In [1]:
!pip install -r ../requirements.txt -qq

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gluonts 0.13.7 requires pydantic~=1.7, but you have pydantic 2.8.2 which is incompatible.
sagemaker-jupyterlab-extension-common 0.1.18 requires pydantic==1.*, but you have pydantic 2.8.2 which is incompatible.


## Import Libraries

In [1]:
from datasets import load_dataset, Dataset
import pandas as pd
from ast import literal_eval
import sys
from pathlib import Path
import os

In [2]:
path_to_src_code = '../'
sys.path.append(path_to_src_code)

In [3]:
from src.utils.functions import *
from src.run_autoprompt import run_autoprompts

## Import Example Dataset

We will load an example dataset from the datasets library that includes a question, ground truth, answer, and contexts. Make sure that ground truths is a list of contexts that are retrieved for the specific question. 

In [11]:
amnesty_qa = load_dataset("explodinggradients/amnesty_qa", "english_v2")
data = amnesty_qa['eval']

Repo card metadata block was not found. Setting CardData to empty.


In [12]:
data.to_pandas()

,question,ground_truth,answer,contexts
0,What are the global implications of the USA Su...,The global implications of the USA Supreme Cou...,The global implications of the USA Supreme Cou...,"[- In 2022, the USA Supreme Court handed down ..."
1,Which companies are the main contributors to G...,"According to the Carbon Majors database, the m...","According to the Carbon Majors database, the m...","[In recent years, there has been increasing pr..."
2,Which private companies in the Americas are th...,The largest private companies in the Americas ...,"According to the Carbon Majors database, the l...",[The issue of greenhouse gas emissions has bec...
3,What action did Amnesty International urge its...,Amnesty International urged its supporters to ...,Amnesty International urged its supporters to ...,"[In the case of the Ogoni 9, Amnesty Internati..."
4,What are the recommendations made by Amnesty I...,The recommendations made by Amnesty Internatio...,Amnesty International made several recommendat...,"[In recent years, Amnesty International has fo..."
5,Who are the target audience of the two books c...,The target audience of the two books created b...,The target audience of the two books created b...,"[In addition to children, parents, teachers, a..."
6,Which right guarantees access to comprehensive...,The right that guarantees access to comprehens...,The right that guarantees access to comprehens...,[The right to truth is a fundamental human rig...
7,Who has the right to be fully informed about h...,The victims of gross human rights violations a...,Everyone has the right to be fully informed ab...,"[In many cases, the identities of perpetrators..."
8,When can individuals be found guilty under Art...,Individuals can be found guilty under Article ...,Under Article 207.3 of the Russian Criminal Co...,[Article 207.3 of the Russian Criminal Code pe...
9,When does the prosecution consider statements ...,The prosecution considers statements contrary ...,Under Article 207.3 of the Russian Criminal Co...,[- As long as their statements are contrary to...


### Ragas

This section goes over how to call the Ragas library on the input dataset. This comes directly from the ragas library: https://github.com/explodinggradients/ragas

In [4]:
from ragas import evaluate
from ragas.metrics import faithfulness, answer_correctness, context_precision, answer_similarity
from langchain_community.chat_models import BedrockChat
from langchain_aws import ChatBedrock
from botocore.client import Config
from langchain_community.embeddings import BedrockEmbeddings

First, initiate Bedrock Chat and Bedrock Embeddings (the large language models used to generate evaluation metrics from Ragas). 

In [12]:
config = {
    "region_name": "us-east-1",  # E.g. "us-east-1"
    "model_id": 'anthropic.claude-3-haiku-20240307-v1:0',  # E.g "anthropic.claude-v2"
    "model_kwargs": {'max_tokens':1000, 'temperature': 0.4,
                                'stop_sequences': ['Question']},
}
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 2})
bedrock_client = boto3.client('bedrock-runtime')

modelId = 'anthropic.claude-3-haiku-20240307-v1:0' # running using claude 3 haiku

bedrock_model = ChatBedrock(region_name=config['region_name'], model_id=modelId, client=bedrock_client, model_kwargs=config['model_kwargs'])
# init the embeddings
bedrock_embeddings = BedrockEmbeddings(
    region_name=config["region_name"],
)

In [13]:
bedrock_model.invoke("hi").content

'Hello! How can I assist you today?'

In this example, we will generate faithfulness, answer correctness, context precision, and answer similarity metrics. Some metrics will fail to parse output but that is ok, this is just to showcase an example of running Ragas on a sample dataset. Experiment with different versions of Bedrock to get more optimal outputs if necessary. 

In [15]:
score = evaluate(data,metrics=[faithfulness,answer_correctness, context_precision, answer_similarity], llm=bedrock_model,
    embeddings=bedrock_embeddings)
score.to_pandas()

Evaluating:   0%|          | 0/80 [00:00<?, ?it/s]

Failed to parse output. Returning None.
Failed to parse output. Returning None.
Exception raised in Job[34]: ValidationError(2 validation errors for ContextPrecisionVerifications
__root__ -> 1 -> reason
  field required (type=value_error.missing)
__root__ -> 1 -> verdict
  field required (type=value_error.missing))
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Fai

,question,ground_truth,answer,contexts,faithfulness,answer_correctness,context_precision,answer_similarity
0,What are the global implications of the USA Su...,The global implications of the USA Supreme Cou...,The global implications of the USA Supreme Cou...,"[- In 2022, the USA Supreme Court handed down ...",NaN,NaN,1.000000,0.943449
1,Which companies are the main contributors to G...,"According to the Carbon Majors database, the m...","According to the Carbon Majors database, the m...","[In recent years, there has been increasing pr...",NaN,NaN,1.000000,0.795071
2,Which private companies in the Americas are th...,The largest private companies in the Americas ...,"According to the Carbon Majors database, the l...",[The issue of greenhouse gas emissions has bec...,1.000000,0.598772,0.833333,0.895087
3,What action did Amnesty International urge its...,Amnesty International urged its supporters to ...,Amnesty International urged its supporters to ...,"[In the case of the Ogoni 9, Amnesty Internati...",1.000000,0.696804,1.000000,0.787217
4,What are the recommendations made by Amnesty I...,The recommendations made by Amnesty Internatio...,Amnesty International made several recommendat...,"[In recent years, Amnesty International has fo...",NaN,NaN,1.000000,0.736213
5,Who are the target audience of the two books c...,The target audience of the two books created b...,The target audience of the two books created b...,"[In addition to children, parents, teachers, a...",0.666667,0.546393,1.000000,0.985571
6,Which right guarantees access to comprehensive...,The right that guarantees access to comprehens...,The right that guarantees access to comprehens...,[The right to truth is a fundamental human rig...,1.000000,0.994421,1.000000,0.977686
7,Who has the right to be fully informed about h...,The victims of gross human rights violations a...,Everyone has the right to be fully informed ab...,"[In many cases, the identities of perpetrators...",NaN,NaN,1.000000,0.921894
8,When can individuals be found guilty under Art...,Individuals can be found guilty under Article ...,Under Article 207.3 of the Russian Criminal Co...,[Article 207.3 of the Russian Criminal Code pe...,0.000000,0.789836,NaN,0.759344
9,When does the prosecution consider statements ...,The prosecution considers statements contrary ...,Under Article 207.3 of the Russian Criminal Co...,[- As long as their statements are contrary to...,1.000000,0.466172,1.000000,0.864686


In [16]:
score.to_pandas().to_csv("../data/ragas_output.csv", index=False)

## Automatic Prompt Generation

Here, automatic prompt generation is done using specific templates (available in instruction_generation_templates folder). Feel free to add more txt files to the list below based on the name of the templates in that folder. 

In [4]:
files = [
    "one-paragraph_instruction_with_demo.txt", 
    "one-sentence_instruction_with_demo.txt", 
    "step-by-step_instruction_with_demo.txt"
]

Run the following command to obtain generated instructions. For a sample dataset of size 20 and 10 candidate instructions generated for each sample (in ../src/utils/config.py NUMBER_OF_PROMPTS=10), it takes about 10 minutes to run. Modify the NUMBER_OF_PROMPTS accordingly. 

In [5]:
%%time
# use the csv file uploaded from Ragas as the input to autoprompt. 
run_autoprompts(True, files, path_to_ragas_outputs="../data/ragas_output.csv")

9


  0%|                                          | 0/27 [00:00<?, ?it/s]

Gen 1:  Here is a system prompt that aims to elicit accurate responses to the given question without referencing the specific details:

Analyze the provided context carefully to understand the key information and requirements. Then, formulate a concise, logical response that directly addresses the question being asked. Draw upon your broa
Gen 2:  Here is a general prompt for a large language model to analyze a given question and context in order to provide an accurate and relevant answer:

Analyze the provided question and context in detail. Carefully consider the intent behind the question, the key information and details present in the context, and any relevant
----------


  7%|██▌                               | 2/27 [00:02<00:32,  1.31s/it]

Draw
Gen 1:  Provide a clear, concise prompt that instructs the AI to analyze the given context, understand the underlying task, and generate a response that accurately and completely answers the question based on the available information, while adhering to the specified guidelines.
Gen 2:  Here is a general prompt for a large language model to analyze a given question and context in order to provide an accurate and relevant answer:

Analyze the provided question and context, then generate a concise, informative, and relevant response that directly addresses the key points raised in the input.
----------


...

Draw
Gen 1:  To solve this task effectively, I will carefully read the provided context, identify the key facts and information relevant to the question, analyze the question to understand its intent, and then synthesize a concise and accurate response that directly addresses the query.
Gen 2:  As an AI assistant, my approach to answering questions based on provided context involves thoroughly analyzing the input, identifying the key facts and details, and synthesizing a clear, informative, and well-reasoned response that directly addresses the question. I am adept at extracting the relevant information,
----------


100%|█████████████████████████████████| 27/27 [01:06<00:00,  2.61s/it]

Draw
Gen 1:  Here is a step-by-step process I would use to tackle this type of task:

1. Carefully read and understand the question being asked. Identify the key information that needs to be addressed in the response.

2. Thoroughly review the provided context, taking note of all
Gen 2:  Here is a general approach I would take to answering questions based on a provided context:

1. Thoroughly read and analyze the given context to fully understand the information and details provided. I will carefully extract all the relevant facts, data, and background details that are necessary to address the question.
----------


100%|█████████████████████████████████| 27/27 [01:09<00:00,  2.57s/it]

Draw
CPU times: user 18.9 s, sys: 812 ms, total: 19.7 s
Wall time: 8min 50s


The output to running the autoprompt function above is stored in the data folder of this repository as 'new_prompts.csv'. In addition, a list of prompts are outputted in the data folder as 'prompt_id.csv'. Below is the output of the best three instructions for each question in the dataset. 

In [6]:
new_prompts = pd.read_csv('../data/new_prompts.csv')

In [7]:
new_prompts.head()

,question,ground_truth,answer,contexts,faithfulness,answer_correctness,context_precision,answer_similarity,best_prompt,second_best_prompt,third_best_prompt
0,When can individuals be found guilty under Art...,Individuals can be found guilty under Article ...,Under Article 207.3 of the Russian Criminal Co...,['Article 207.3 of the Russian Criminal Code p...,0.000000,0.789836,NaN,0.759344,Imagine you are designing a system prompt for ...,You are asked to generate a system prompt for ...,Imagine you are tasked with designing a system...
1,When does the prosecution consider statements ...,The prosecution considers statements contrary ...,Under Article 207.3 of the Russian Criminal Co...,['- As long as their statements are contrary t...,1.000000,0.466172,1.000000,0.864686,You are an AI assistant created to help people...,Pretend you are an expert witness or legal ana...,You are a researcher or journalist investigati...
2,What factors have contributed to the decline o...,The factors that have contributed to the decli...,There are several factors that have contribute...,['The economic challenges facing Nicaragua hav...,NaN,0.235575,1.000000,0.942299,You are a human rights researcher investigatin...,You are an AI assistant tasked with generating...,You are an expert in analyzing political and s...
3,What conditions designate wetlands as Ramsar s...,The conditions that designate wetlands as Rams...,Wetlands are designated as Ramsar sites based ...,['Wetlands designated as Ramsar sites must mee...,0.555556,0.458345,1.000000,0.910301,You are an AI assistant tasked with generating...,You are an AI engineer tasked with developing ...,You are an AI developer tasked with designing ...
4,Where was COP15 held in 2022?,"COP15 was held in Montreal, Canada in 2022.","COP15 was held in Kunming, China in 2022.","['The city of Kunming, located in the Yunnan p...",1.000000,0.209465,0.583333,0.837860,You are an AI assistant tasked with providing ...,You are an AI assistant tasked with providing ...,You are an AI assistant created to provide inf...


Below are the prompts/instructions mapped to ids. 

In [8]:
prompt_ids = pd.read_csv('../data/prompt_ids.csv')

In [9]:
prompt_ids.head()

,Id,Prompts
0,0,Imagine you are designing a system prompt for ...
1,1,You are an AI assistant created to help people...
2,2,You are a human rights researcher investigatin...
3,3,You are an AI assistant tasked with generating...
4,4,You are an AI assistant tasked with providing ...
